In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, precision_score, accuracy_score,f1_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import Pipeline
from nltk.corpus import names
from collections import defaultdict
import re

%matplotlib inline

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data = pd.Series(newsgroups.data)
df = pd.DataFrame(data)
df.columns = ['text'] + df.columns.tolist()[1:]
df['target'] = pd.Series(newsgroups.target)
df.head()

,text,target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4


In [3]:
print('shape of dataframe: ', df.shape)
print('number of target variables: ',df.target.nunique())
print('null target variables: ', df.target.isna().any())
print('null text: ',df.text.isna().any())

shape of dataframe:  (18846, 2)
number of target variables:  20
null target variables:  False
null text:  False


In [4]:
#remove emails
df['text'] = df['text'].apply(lambda row: re.sub(r"\S*@\S*\s?","",row))

#remove extra spaces
df['text'] = df['text'].apply(lambda row: re.sub(r"\s+"," ",row))

# #remove single quote marks
df['text'] = df['text'].apply(lambda row: re.sub(r"\'","",row))

#make all text lower case
df['text'] = df['text'].str.lower()

In [5]:
#instatiate tokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')

#tokenize test
df['tokenized_text'] = df.apply(lambda row: tokenizer.tokenize(row['text']),axis=1)

#define stop words
stop_words=set(stopwords.words("english"))

#remove stop words
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [item for item in x if item not in stop_words])

#instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

#lemmatize text
df['lemmatized'] = df['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
df.lemmatized = df.lemmatized.apply(lambda x: " ".join(x) )

df.sample(5)

,text,target,tokenized_text,lemmatized
10706,"sorry, but i just wanted to be the first hypoc...",18,"[sorry, wanted, first, hypocrite, say, hope, i...",sorry wanted first hypocrite say hope im late ...
5306,"ive been using version 2.5.2 of ghostscript, a...",2,"[ive, using, version, 2, 5, 2, ghostscript, im...",ive using version 2 5 2 ghostscript im quite s...
9118,[...] note that i _never_ said that depressio...,0,"[note, never, said, depression, destruction, n...",note never said depression destruction nuclear...
14641,this is just basicly right. the connections t...,12,"[basicly, right, connections, cannot, handle, ...",basicly right connection cannot handle touchto...
11739,hi! anyone knows how can i change an icon fore...,2,"[hi, anyone, knows, change, icon, forever, eve...",hi anyone know change icon forever ever mean p...


In [6]:
df.tokenized_text.isna().any()

False

In [7]:
#train test validation split

X,y = df.lemmatized,df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [8]:
#pipeline for logistic regression

pipe_logreg = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

parameters_logreg = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [0.5],
#     'tfidf__max_features': [None, 5000, 10000, 50000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
    "clf__C": [1],
    "clf__class_weight": ['balanced'],
    "clf__solver": ['newton-cg'],
    "clf__l1_ratio":[0,0.2,0.4,0.6,0.8,1],
    "clf__multi_class":['multinomial']
}

gs_logreg = GridSearchCV(estimator=pipe_logreg,
            param_grid=parameters_logreg,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_logreg.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_logreg.best_score_)

# Best params
print('\nBest params:\n', gs_logreg.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.8min finished
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


Best accuracy: 0.708

Best params:
 {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__l1_ratio': 0, 'clf__multi_class': 'multinomial', 'clf__solver': 'newton-cg', 'tfidf__max_df': 0.5, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 1)}


In [ ]:
#pipeline for decision tree

pipe_tree = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])
parameters_tree = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
    "clf__criterion": ["gini", "entropy"],
    "clf__max_depth":[50,75,100],
    "clf__min_samples_split" : [100,150,200]
}

gs_tree = GridSearchCV(estimator=pipe_tree,
            param_grid=parameters_tree,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_tree.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_tree.best_score_)

# Best params
print('\nBest params:\n', gs_tree.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.0min


In [ ]:
#pipeline for random forest

pipe_rf = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])
parameters_rf = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
    "clf__criterion": ["gini", "entropy"],
    "clf__max_depth":[50,75,100],
    "clf__min_samples_split" : [100,150,200]
}

gs_rf = GridSearchCV(estimator=pipe_rf,
            param_grid=parameters_rf,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_rf.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_rf.best_score_)

# Best params
print('\nBest params:\n', gs_rf.best_params_)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier

In [ ]:
#pipeline for NaiveBayes



In [ ]:
#pipeline for xgboost

pipe_xgb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svd',   TruncatedSVD(algorithm='randomized')),
    ('clf',   XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20)),
])

parameters_xgb = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or 
    "svd__n_components": [250,500,750,1000],
    "clf__n_estimators": [500,1000,5000],
    "clf__learning_rate":[0.01,0.1,1],
    "clf__min_samples_split" : [100,150,200]
}

gs_xgb = GridSearchCV(estimator=pipe_xgb,
            param_grid=parameters_xgb,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_xgb.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_xgb.best_score_)

# Best params
print('Best accuracy: %.3f' % gs_xgb.best_score_)